In [1]:
import run
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


import numpy as np

from track import Track
from environment import Car, Environment
from sys import argv
import os
import sys

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


# track = Track.read_from(argv[1])
# env = Environment(track)
tracks_path = ['../tracks/100.track','../tracks/101.track','../tracks/102.track']

tracks = [Track.read_from(x) for x in tracks_path]
envs = [Environment(x) for x in tracks]

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.l1 = nn.Linear(4, 32)
#         self.b1 = nn.BatchNorm1d(8)
        self.l2 = nn.Linear(32,16)
#         self.b2 = nn.BatchNorm1d(16)
#        self.l3 = nn.Linear(16,64)
    
        self.l3 = nn.Linear(16,4)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu((self.l1(x)))
        x = F.relu((self.l2(x)))
        x = F.relu((self.l3(x)))
   #     x = F.relu(self.l4(x))
        return x

class DQN1(nn.Module):
    def __init__(self):
        super(DQN1, self).__init__()
        self.l1 = nn.Linear(4, 4)


    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu((self.l1(x)))
        return x
    
def get_input(i):
    return envs[i].state()


def send_action(i):
    broken, done, progress = envs[i].step(action)
    return (broken, done, progress)

BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.95
EPS_END = 0.2
EPS_DECAY = 200
TARGET_UPDATE = 100

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
# init_screen = get_screen()
# _, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = 4

policy_net = DQN()
policy_net.eval()
target_net = DQN()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
#    print(eps_threshold)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]],  dtype=torch.long)


episode_durations = []
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)),  dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
#     sys.stdout.write(f'\r Loss= {loss} \n val_proc= {val_proc}')

    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
state_proc = lambda state: torch.tensor(torch.from_numpy(np.array(state)).unsqueeze(0), dtype=torch.float32)
num_episodes = 500
last_val = 0
val_proc = 0
for i in range(100):
    for track in tracks:
        env = Environment(track)
        for i_episode in range(num_episodes):
            # Initialize the environment and state
            last_screen = state_proc(env.state())
            current_screen = state_proc(env.state())
            state = current_screen
            for t in count():

                # Select and perform an action
                action = select_action(state)
                broken, done, reward = env.step(action.item())
                reward = torch.tensor([reward*100])

                # Observe new state
                last_screen = current_screen
                current_screen = state_proc(env.state())
                if not done and not broken:
                    next_state = current_screen
                else:
                    next_state = None

                # Store the transition in memory
                if (next_state is not None):
                    memory.push(state, action, next_state, reward)

                # Move to the next state
                state = next_state

                # Perform one step of the optimization (on the target network)
                optimize_model()
                if done or broken:
                    episode_durations.append(t + 1)
                    break
            # Update the target network, copying all weights and biases in DQN
            if i_episode % TARGET_UPDATE == 0:
                val_proc = 0
                for j in range(3):
                    env = Environment(tracks[1])
                    vbroken = False
                    vdone = False
                    vlast_screen = state_proc(env.state())
                    vcurrent_screen = state_proc(env.state())
                    vstate = vcurrent_screen - vlast_screen
                    while not vbroken and not vdone:
                        vaction = select_action(vstate)
                        vbroken, vdone, vprogress = env.step(vaction.item())
                        vstate = state_proc(env.state())
                    val_proc += vprogress
                if (val_proc >= last_val):
                    torch.save(policy_net.state_dict(),'weights.ckp')
                target_net.load_state_dict(policy_net.state_dict())
                sys.stdout.write(f'\r val_proc= {val_proc}')

            

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


 val_proc= 0.142857142857142856

In [17]:
env = Environment(tracks[1])
broken = False
done = False
last_screen = state_proc(env.state())
current_screen = state_proc(env.state())
state = current_screen - last_screen
while not broken and not done:
    action = select_action(state)
    broken, done, progress = env.step(action.item())
    state = state_proc(env.state())
print(progress)

0.047619047619047616


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


In [33]:
torch.save(policy_net.state_dict(),'weights.ckp')